In [1]:
import pandas as pd 
import numpy as np 
import pickle 
from sklearn.model_selection import train_test_split

In [ ]:
# Logical - easy
with open(f'data/datasets/coherence/curated_dataset_full.pkl', 'rb') as file:
    curated_dataset = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/datasets/curated_dataset_full.pkl'

In [56]:
def get_data_split(task: str, curated_dataset, probe_config, other_dataset=None, cutoff=1500):


    if task in ['negation', 'disjunction', 'conjunction']:
        # In this case curated dataset is the logical dataset + the remainder
        remainder = curated_dataset[~curated_dataset['filename'].isin(['common_claim_true_false.csv', 'companies_true_false.csv', 'counterfact_true_false.csv'])]
        curated_dataset = pd.concat([remainder, other_dataset])
        train_set, test_set = train_test_split(curated_dataset, test_size=0.2, random_state=42, stratify=curated_dataset['filename'])
        test_df = test_set.dropna()
        
    elif task == 'inference':
        remainder = curated_dataset[~curated_dataset['filename'].isin(['common_claim_true_false.csv', 'companies_true_false.csv', 'counterfact_true_false.csv',
                                                                      'cities.csv', 'cities_cities_conj.csv', 'cities_cities_disj.csv', 'neg_cities.csv'
                                                                      ])]
        curated_dataset = pd.concat([remainder, other_dataset])
        train_set, test_set = train_test_split(curated_dataset, test_size=0.2, random_state=42, stratify=curated_dataset['filename'])
        test_df = test_set.dropna()
        

    # Trim for batch size
    train_set = train_set.iloc[:-(len(train_set) % probe_config), :]
    X_clean_train = list(train_set['statement'])
    y_clean_train = list(train_set['label'])
    
    return (X_clean_train, y_clean_train, test_df)